In [1]:
import torch
from torch.functional import F
from torch import nn, optim
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, datasets
import tqdm
from statistics import mean

import glob
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self,data,transform=None):
        self.transform = transform
        self.data = data
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        img_l = self.data[idx]
        img_l = img_l/255
        
        label = 1
        return img_l.transpose(2,0,1),label
    
class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim,activation):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.activation = activation
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out
    
    
class Reshape(nn.Module):
    def __init__(self,c_size,h_size,w_size):
        super().__init__()
        self.c_size = c_size
        self.h_size = h_size
        self.w_size = w_size
    def forward(self, x):
        return x.view(x.shape[0], self.c_size,self.h_size,self.w_size)
    
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0],x.shape[1]*x.shape[2]*x.shape[3])

In [3]:
class sa_Generator(nn.Module):
    def __init__(self,sablock):
        super().__init__()
        
        self.fc_layer = nn.Linear(100,512*4*4)
        self.reshape  = Reshape(512,4,4)
        self.layer1 = nn.Sequential(nn.ConvTranspose2d(512,256,4,2,1,bias=False),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(inplace=True)
                                    )
        
        self.layer2 = nn.Sequential(nn.ConvTranspose2d(256,128,4,2,1,bias=False),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(inplace=True))
        
        self.layer3 = nn.Sequential(nn.ConvTranspose2d(128,64,4,2,1,bias=False),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(inplace=True),
                                    sablock(64,"relu"))
        
        self.layer4 = nn.Sequential(nn.ConvTranspose2d(64,32,4,2,1,bias=False),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(inplace=True),
                                    sablock(32,"relu"))
        
        self.layer5 = nn.Sequential(nn.ConvTranspose2d(32,3,4,2,1,bias=False))
        self.tanh = nn.Tanh()
        
        
    def forward(self, x):
        x = self.fc_layer(x)
        x = self.reshape(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)  
        x = self.tanh(x)
        return x

class sa_Discriminator(nn.Module):
    def __init__(self,sablock):
        super().__init__()
        self.layer1 = nn.Sequential(nn.utils.spectral_norm(nn.Conv2d(3,16,4,2,1,bias=False)),
                                    nn.LeakyReLU(inplace=True)
                                    )
        self.layer2 = nn.Sequential(nn.utils.spectral_norm(nn.Conv2d(16,32,4,2,1,bias=False)),
                                    nn.LeakyReLU(inplace=True)
                                    )
        self.layer3 = nn.Sequential(nn.utils.spectral_norm(nn.Conv2d(32,64,4,2,1,bias=False)),
                                    nn.LeakyReLU(inplace=True)
                                    )
        self.layer4 = nn.Sequential(nn.utils.spectral_norm(nn.Conv2d(64,128,4,2,1,bias=False)),
                                    nn.LeakyReLU(inplace=True),
                                    sablock(128,"relu")
                                    )
        self.layer5 = nn.Sequential(nn.utils.spectral_norm(nn.Conv2d(128,256,4,2,1,bias=False)),
                                    nn.LeakyReLU(inplace=True),
                                    sablock(256,"relu"),
                                    Flatten()
                                    )
        
        self.fc = nn.Linear(256*4*4,2)
        
#         nn.init.xavier_normal_(self.fc.weight.data)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc(x)
        return x

In [4]:
device = "cuda"
batch_size = 32
model_G = sa_Generator(Self_Attn).to(device)
model_D = sa_Discriminator(Self_Attn).to(device)

model_G = torch.nn.DataParallel(model_G,device_ids=[0,1])
model_D = torch.nn.DataParallel(model_D,device_ids=[0,1])

loss_f = nn.CrossEntropyLoss().to(device)

params_G = optim.Adam(model_G.parameters(),lr=0.0001,betas=(0.5, 0.99))
params_D = optim.Adam(model_D.parameters(),lr=0.0001,betas=(0.5, 0.99))

ones = torch.ones(batch_size*2).to(device).long() # 正例 1
zeros = torch.zeros(batch_size*2).to(device).long() # 負例 0

In [5]:
data = np.load("train_anime_128128.npy")
data = torch.FloatTensor(data/255)
test_z = torch.randn(36, 100).to(device)

In [ ]:
dis_num = 2

for epoch in range(1000):
    losses_G = []
    losses_D = []
    rnd = np.random.permutation(data.shape[0])
    for i,batch in enumerate(range(0,data.shape[0],batch_size*dis_num)): 
        start_idx = ((batch_size*dis_num)*i)
        end_idx   = ((batch_size*dis_num)*(i+1))
        img = data[rnd[start_idx:end_idx]]
        for dis_n in range(dis_num):
            
            real_img = img[batch_size*dis_n:batch_size*(dis_n+1)]
            b_size = real_img.shape[0]
            z = torch.randn(b_size, 100).to(device)
            fake_img = model_G(z)
            fake_img_tensor = fake_img.detach()

            ##discriminator loss

            real_img = real_img.float()
    #         real_img = (real_img*2)-1
            real_img = real_img.to(device)

            real_out = model_D(real_img)

    #         loss_D_real = torch.nn.ReLU()(1.0 - real_out).mean()
            loss_D_real = loss_f(real_out, ones[: b_size])

            fake_out = model_D(fake_img_tensor)
            ##discriminator_fake loss
    #         loss_D_fake = torch.nn.ReLU()(1.0 + fake_out).mean()
            loss_D_fake = loss_f(fake_out, zeros[: b_size])
            loss_D = loss_D_real + loss_D_fake

            losses_D.append(loss_D.item())

            model_D.zero_grad()
            model_G.zero_grad()
            loss_D.backward()
            params_D.step()
        

        #Genrator
        z = torch.randn(b_size, 100).to(device)
        fake_img = model_G(z)
        fake_img_tensor = fake_img.detach()

        out = model_D(fake_img)
#         loss_G = - out.mean()
        loss_G = loss_f(out, ones[: b_size])
        losses_G.append(loss_G.item())

        model_D.zero_grad()
        model_G.zero_grad()
        loss_G.backward()
        params_G.step()
        
    if (epoch%10) == 0:
        print("Genrator",sum(losses_G)/len(losses_G))
        print("Discriminator",sum(losses_D)/len(losses_D))
        with open("log.txt","a",) as w:
            a = "-"*50
            w.write(f"{a}\n{epoch}epoch\nGenerator: {sum(losses_G)/len(losses_G)}\nDiscriminator: {sum(losses_D)/len(losses_D)} \n")

        ##img_plot
        test_fake_img = model_G(test_z)
        fig = plt.figure(figsize=(20,20))
        for i,img_array in enumerate(test_fake_img.detach().cpu().numpy()):
            if i == 36:
                break
            ax1 = fig.add_subplot(6,6,(i+1))
            img_array = img_array.transpose(1,2,0)
            ax1.imshow(img_array)
        plt.savefig(f"./result/{epoch}epoch_sa.jpg")
        plt.close()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Genrator 1.6366742135996515
Discriminator 0.6633828321194395


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Genrator 1.5652759503176872
Discriminator 0.862005039098415


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Genrator 5.228645093897556
Discriminator 0.09963066881879214


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Genrator 5.286721153462187
Discriminator 0.11315021327538217


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i